In [1]:
import numpy as np
import shap
n_features = 'all'
term_pred = 'short_term'
data_file_name = 'data_y10'
endpt = 10
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
training_id_file_name = 'all_training_ID_outerloop_cohort_0_10'
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'
work_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2'

In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:
import pkg_resources
#pkg_resources.require("scikit-learn==1.0.2")
import sklearn

import sksurv

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from numpy.random import seed

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

In [4]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [5]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
feature_name_list = feature_space.columns.values


In [6]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,3689.025,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,3689.025,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,3689.025,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100048415270,0,1169.000,0,1,137.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,18,0.008850
4,100056526386,0,3689.025,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4310,416771521620,0,3689.025,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4311,416783315386,0,3689.025,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4312,416796224310,0,3689.025,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [8]:
def f_rsf_input_as_matrix(X):
    surv = rsf.predict_survival_function(X, return_array=True)
    last_yr_pred_prob_surv=surv[:,-1]
    return last_yr_pred_prob_surv


def vimp_shap_tabular(feature_names, vals):
    feature_importance_df = pd.DataFrame(list(zip(feature_names, vals)), columns=['variable_name','vimp'])
    feature_importance_df.sort_values(by=['vimp'], ascending=False,inplace=True)
    feature_importance_df['normalized_vimp'] = (feature_importance_df['vimp'] - min(feature_importance_df['vimp']))/(max(feature_importance_df['vimp'])-min(feature_importance_df['vimp']))
    
    # merge with variable description:
    var_dict = pd.read_csv(load_dir+'/y5_all_vars_dictionary_manually_selecting_vars (3).csv', encoding= 'unicode_escape')
    var_dict[['Variable.Name', 'Variable.Label']]
    # merge vimp table with var dict to get var description:
    feature_importance_df_with_vardes = feature_importance_df.merge(
        var_dict[['Variable.Name', 'Variable.Label']], how = 'inner', right_on= 'Variable.Name', left_on = 'variable_name', copy= False)
    return(feature_importance_df_with_vardes[['Variable.Name','Variable.Label','normalized_vimp','vimp']])


In [9]:
# Outerloop:
c_index = np.zeros((25,1))
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']

    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values

    y_train = sksurv.util.Surv.from_dataframe('event', 'time', train_df)
    y_test = sksurv.util.Surv.from_dataframe('event', 'time', test_df)

    ##########################
    # Build model:


    seed(1)
    rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=20)
    rsf.fit(featurespace_train_df, y_train) 

    rsf.score(featurespace_test_df, y_test) 
    c_index[fold] = rsf.score(featurespace_test_df, y_test) 
    
    savedir = os.path.join(work_dir,'csv_files/rsf/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    

    ## SHAP:
    
    X_train_df_summary = shap.kmeans(featurespace_train_df, 10)
    explainer_kmeans = shap.KernelExplainer(f_rsf_input_as_matrix, X_train_df_summary, feature_names=feature_name_list) 
    kmeans_shaps = explainer_kmeans.shap_values(featurespace_test_df)

    vimp_shap_kmeans = vimp_shap_tabular(feature_name_list, np.abs(kmeans_shaps).mean(0))
    
    vimp_shap_kmeans.to_csv(savedir+'/shap_kernel_explainer_fold_'+str(actual_fold)+'.csv', index = None, header = True)



FOLD 0:





FOLD 1:





FOLD 2:





FOLD 3:





FOLD 4:



...



FOLD 20:





FOLD 21:





FOLD 22:





FOLD 23:





FOLD 24:



In [10]:
c_index_df = pd.DataFrame(c_index)
c_index_df.to_csv(savedir+'/c_index_all_folds'+'.csv', index = None, header = True)

